In [ ]:
import copy
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os.path
import numpy as np
import pickle
import scipy as sp
from skimage.color import rgb2hsv, hsv2rgb, xyz2rgb
from skimage import data, exposure, filters, img_as_float, io


from lippmann import show_spectrum, show_lippmann_transform, lippmann_transform
from display_spectral_data import load_specim_data, read_file
from color_tools import upsample_hue_saturation, from_spectrum_to_xyz, from_xyz_to_rgb
from spectrum import Spectrum3D
import spectrum_recovery_pool as recov

### Load and inspect the resutls of the spectral recovery 

`name` file name prefix, the same for measured and recovered data

`directory` name of the directory in which the results have been stroed

In [ ]:
name = "other_color_checker" 
directory = "color_checker_2021" 
results_path = f"PNAS/{directory}/{name}"

c0 = 299792458 #TODO import this 
n0 = 1.5
c = c0 / n0

result = np.load(f"{results_path}.npy")
result = np.clip(result, a_min = 0, a_max = np.inf)

with open(f"{results_path}.pkl", "rb") as pickled_params:
    params = pickle.load(pickled_params)

In [ ]:
print("Experiment setup")

Z = params["Z"]
k0 = params["k0"]

if "estimate_depth" in params:
    if params["estimate_depth"]:
        plt.matshow(params["Z_estimates"])
        plt.colorbar()
        plt.savefig(f"{results_path}_z.pdf")
        plt.show()
        params.pop("Z_estimates")
        params.pop("k0_estimates")
        Z = np.median(params['Z_estimates'])
        k0 = np.median(params["k0_estimates"])
        print(f"median depth {Z:.2e}")
        print(f"median decay {k0}")
print_params = copy.copy(params)
print_params.pop("errors")
print_params

In [ ]:
plt.matshow(params["errors"])
plt.colorbar()
plt.show()

### Load data from specim camera
Stored in `Cubes`

In [ ]:
downsampled, wavelengths = load_specim_data("Cubes/" + name, params["downsampling"], cut=True)

omegas_rec = 2 * np.pi * c / wavelengths
if params["visible"]: #TODO what should be there? 
    omegas_est = np.linspace(2 * np.pi * c / 400E-9, 2 * np.pi * c / 700E-9, result.shape[2])
    print("Restricting spectrum to visible")
else:   
    omegas_est = np.linspace(np.max(omegas_rec), np.min(omegas_rec), result.shape[2])
wavelengths_est = 2 * np.pi * c / omegas_est

### Correct for dyes, if proper file exit

We know the dye profile only for the plate whe have made ourselves, i.e. color checker

In [ ]:
if os.path.isfile(f'Cubes/{name}_Transmission_Reference.txt'):

    w, trans, _ = read_file(f'Cubes/{name}_Transmission_Reference.txt')
    w, trans_glass, _ = read_file(f'Cubes/{name}_Transmission_OnlyGlass.txt')
    dyes = (trans_glass - trans)

    dyes_interpolated = np.interp(wavelengths_est, w, dyes)
    dyes_interpolated = dyes_interpolated/np.max(dyes_interpolated)
    show_spectrum(wavelengths_est, dyes_interpolated, show_background=True, short_display=True, visible=False)
    plt.title("dye profile")
    plt.show()

    dye_corrected = result / dyes_interpolated[None, None, ]
else:
    dye_corrected = result

### Show the recorded and reconstructed colors of the plate

In [ ]:
def spectrum_to_rgb(wavelengths, spectrum):
    spectrum_xyz = from_spectrum_to_xyz(wavelengths, spectrum, normalize=False)
    spectrum_xyz = spectrum_xyz / np.min(np.sum(spectrum_xyz, axis=2))
    return from_xyz_to_rgb(spectrum_xyz)


recorded_rgb = spectrum_to_rgb(wavelengths, downsampled)
estimated_rgb = spectrum_to_rgb(wavelengths_est, dye_corrected)
pixel = (70, 80)
true_pixel = (25, 25)

fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(10, 5))

true_path = "Cubes/" + name + "_true"
if os.path.isfile(true_path + ".dat"):
    ground_truth, _ = load_specim_data(true_path, params["downsampling"], cut=True)
    ground_truth_rgb = spectrum_to_rgb(wavelengths, ground_truth)
    ax0.imshow(ground_truth_rgb)
    true_rect = Rectangle((true_pixel[1] - 0.5, true_pixel[0] - 0.5), 1, 1, alpha=1, color="none", ec="white", lw=2,zorder=10)
    ax0.add_patch(true_rect)
ax1.imshow(recorded_rgb)
ax2.imshow(estimated_rgb)
ax0.set_title("Ground trugh (RGB)")
ax1.set_title("Recorded")
ax2.set_title("Reconstructed")
rect = Rectangle((pixel[1] - 0.5, pixel[0] - 0.5), 1, 1, alpha=1, color="none", ec="white", lw=2,zorder=10)
rect2 = Rectangle((pixel[1] - 0.5, pixel[0] - 0.5), 1, 1, alpha=1, color="none", ec="white", lw=2,zorder=10)
ax1.add_patch(rect)
ax2.add_patch(rect2)
plt.show()

plt.imsave(results_path + "_estimated.png", estimated_rgb)

### Inspect spectra at a single pixel 
Pixel is choosen in the cella bove an visualised as a white box

In [ ]:
fig, (ax0, ax1, ax2, ax3, ax4) = plt.subplots(1, 5, figsize=(15, 3))

show_spectrum(wavelengths, downsampled[pixel], show_background=True, short_display=True, ax=ax1, visible=True)
show_spectrum(wavelengths_est, dye_corrected[pixel], show_background=True, short_display=True, ax=ax2, visible=True)

depths = np.linspace(0,Z,200)
show_lippmann_transform(depths, lippmann_transform(wavelengths_est, result[pixel], depths, r=params["r"], k0=k0)[0], ax=ax3, short_display=True)

A = recov.generate_matrix_A(omegas_est, Z, r=params["r"], k0=k0)
show_spectrum(wavelengths_est, np.abs(A @ result[pixel])**2, show_background=True, short_display=True, ax=ax4, visible=True)
if os.path.isfile(true_path + ".dat"):
    show_spectrum(wavelengths, ground_truth[true_pixel], show_background=True, short_display=True, ax=ax0, visible=True)
ax0.set_title("Ground truth")    
ax1.set_title("Reflected spectrum")
ax2.set_title("Reconstruced spectrum")
ax3.set_title("Pattern")
ax4.set_title("Re-estimated reflected spectrum")
plt.show()

### Upsample recorded and reconstructed images to full resolution
Using a .png photo of the plate, run only if `downsampling` is not one

this is only a visualisation tool, so adjust `gamma` and `gain` for the images to look good:

In [ ]:
if params["downsampling"] > 1:
    image = io.imread("Cubes/" + name + ".png")
    image = np.swapaxes(image[:, ::-1, :3], 1, 0)
    if os.path.exists("Cubes/" + name + "_cut.txt"):
        cut_idx = np.loadtxt("Cubes/" + name + "_cut.txt").astype(np.int)
        image = image[cut_idx[0, 0]:cut_idx[0, 1], cut_idx[1, 0]:cut_idx[1, 1]]

    gamma = 0.5
    gain = 1.5
    image = exposure.adjust_gamma(image, gamma, gain)

    recorded_upsampled = upsample_hue_saturation(image, recorded_rgb, order=1)
    estimated_upsampled = upsample_hue_saturation(image, estimated_rgb, order=1)

    fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(10, 5))
    ax0.imshow(image)
    ax0.set_title("Photo of a plate")
    ax1.imshow(recorded_upsampled)
    ax1.set_title("Upsampled recording")
    ax2.imshow(estimated_upsampled)
    ax2.set_title("Upsampled reconstruction")
    plt.show()
    plt.imsave(results_path + "_estimated_upsampled.png", estimated_upsampled)